#Information
Group 9 : Resume parser

Content : Using TF-IDF to check similarity of resumes based on job description

In [ ]:
# packages
import pandas as pd
import numpy as np
import seaborn as sns
import re
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Move the directory to the csv files
%cd /content/drive/MyDrive/NLP_Final_changes/

/content/drive/MyDrive/NLP_Final_changes


In [ ]:
ls

Bert_classification.ipynb                 sample_java_job_description.docx
Bert_Similarity.ipynb                     sample_java_resumes.csv
Category_plot.png                         sample_testing_job_description.docx
cleaned_resume_final.csv                  sample_testing_resumes.csv
Preprocessing.ipynb                       TF_IDF_Classification.ipynb
resume_classification_model/              TF_IDF_Similarity.ipynb
ResumeDataSet.csv                         Word2Vec.ipynb
sample_data_science_job_description.docx  Word_cloud.png
sample_data_science_resumes.csv


In [ ]:
# Same preprocessing function used while preprocessing the resumes
# Preprocessing the data
def preprocess_text(sentence_list):
  lines_list = []
  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()

  for line in sentence_list:
      line = line.lower()
      words=[]
      for word in word_tokenize(line):
        if word.isalpha():
          if word not in stop_words:
            words.append(lemmatizer.lemmatize(word))
      lines_list.append(' '.join(words))
  
  return lines_list

#Sample Data Science resumes and job description

In [ ]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_data_science_resumes.csv")

In [ ]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [ ]:
df.head()

,Category,Resume,cleaned_resume
0,Data Science,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Data Science,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Data Science,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Data Science,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [ ]:
# Package required to copy the data from word documents
!pip install docx2txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=55d215378719370e2847326c7c1f0936c9b2c182df66c9de1716e4fb9531a9a1
  Stored in directory: /root/.cache/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt


In [ ]:
import docx2txt

In [ ]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_data_science_job_description.docx")

In [ ]:
# Preprocess job description
job_description_tokenised = preprocess_text([job_description])

In [ ]:
# import packages to use cosine similarity

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [ ]:
# Combine job description and the resume in a list which we need to find cosine similarity of
# Later vectorize the data and pass it to cosine similarity

def get_similarity(resume_list):
  similarity_list = []
  for i in resume_list:
    combined_text = [job_description_tokenised[0], i]
    final_vector = vectorizer.fit_transform(combined_text)
    similarity_list.append(round(cosine_similarity(final_vector)[0][1]*100,2))

  return similarity_list

In [ ]:
# Get the similarity of each resume based on job description

pd.set_option('mode.chained_assignment', None)
df['Similarity_Percent'] = get_similarity(df['cleaned_resume'])

In [ ]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [ ]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [ ]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
19,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,41.86,1
16,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,39.15,2
8,Data Science,Hani Haider\nPhone: 781-491-3977 | Email: haid...,hani haider phone email boston linkedin github...,35.77,3
7,Data Science,VIGNAN VENNAMPALLY vennampally.v@northeastern....,vignan vennampally linkedin github education n...,26.16,4
10,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,24.23,5


In [ ]:
import random

In [ ]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [ ]:
# Shuffle the list
random.shuffle(manual_rank)

In [ ]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
19,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,41.86,1,19
16,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,39.15,2,4
8,Data Science,Hani Haider\nPhone: 781-491-3977 | Email: haid...,hani haider phone email boston linkedin github...,35.77,3,5
7,Data Science,VIGNAN VENNAMPALLY vennampally.v@northeastern....,vignan vennampally linkedin github education n...,26.16,4,20
10,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,24.23,5,11
6,Data Science,"Nikitha Kanaparthi\nBoston, MA. 02120 • +1 (20...",nikitha kanaparthi boston linkedin github educ...,23.20,6,18
13,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,20.73,7,16
4,Data Science,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...,19.63,8,8
18,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skill ability quickly grasp technical...,19.53,9,6
17,Data Science,Education Details \r\n B.Tech Rayat and Bahr...,education detail rayat bahra institute enginee...,17.02,10,12


In [ ]:
# Install rbo to use the rbo ranking similarity
!pip install rbo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import rbo

In [ ]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.5045035163494915

#Sample Java resumes and job description

In [ ]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_java_resumes.csv")

In [ ]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [ ]:
df.head()

,Category,Resume,cleaned_resume
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Java Developer,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Java Developer,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Java Developer,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [ ]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_java_job_description.docx")

In [ ]:
# Preprocess job description
job_description_tokenised = preprocess_text([job_description])

In [ ]:
# Get the similarity of each resume based on job description

pd.set_option('mode.chained_assignment', None)
df['Similarity_Percent'] = get_similarity(df['cleaned_resume'])

In [ ]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [ ]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [ ]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,36.62,1
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,35.57,2
15,Java Developer,Education Details \r\nJanuary 2016 B.E Informa...,education detail january information technolog...,30.86,3
18,Java Developer,Skills â¢ Language: Java â¢ Operating System...,skill language java operating system window li...,24.39,4
11,Java Developer,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,...",technical skill skill java sql c bootstrap jsp...,23.88,5


In [ ]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [ ]:
# Shuffle the list
random.shuffle(manual_rank)

In [ ]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,36.62,1,5
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,35.57,2,8
15,Java Developer,Education Details \r\nJanuary 2016 B.E Informa...,education detail january information technolog...,30.86,3,17
18,Java Developer,Skills â¢ Language: Java â¢ Operating System...,skill language java operating system window li...,24.39,4,1
11,Java Developer,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,...",technical skill skill java sql c bootstrap jsp...,23.88,5,12
13,Java Developer,"Operating Systems Windows XP, 7, 10. Tools/Pac...",operating system window xp visual basic uml pa...,19.93,6,3
14,Java Developer,"Computer Skills: Languages And Script: JSP, Se...",computer skill language script jsp servlet htm...,18.26,7,19
12,Java Developer,Education Details \r\nJanuary 2013 Master of E...,education detail january master engineering in...,17.23,8,10
3,Java Developer,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,16.93,9,18
17,Java Developer,SKILLS: - 1) Team leading 2) Self-motivated 3)...,skill team leading hard working strength ready...,15.57,10,2


In [ ]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.5686869668736101

#Sample testing Rsumes and job description

In [ ]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_testing_resumes.csv")

In [ ]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [ ]:
df.head()

,Category,Resume,cleaned_resume
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Testing,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Testing,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Testing,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [ ]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_testing_job_description.docx")

In [ ]:
# Preprocess job description
job_description_tokenised = preprocess_text([job_description])

In [ ]:
# Get the similarity of each resume based on job description

pd.set_option('mode.chained_assignment', None)
df['Similarity_Percent'] = get_similarity(df['cleaned_resume'])

In [ ]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [ ]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [ ]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
18,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,24.78,1
13,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,24.78,2
16,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,21.56,3
15,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,21.56,4
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,20.18,5


In [ ]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [ ]:
# Shuffle the list
random.shuffle(manual_rank)

In [ ]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
18,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,24.78,1,10
13,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,24.78,2,15
16,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,21.56,3,5
15,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,21.56,4,17
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,20.18,5,9
14,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",personal skill quick learner eagerness learn n...,18.53,6,2
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,14.59,7,6
12,Testing,Computer Skills: â¢ Proficient in MS office (...,computer skill proficient m office word basic ...,14.03,8,19
10,Testing,â¢ Good logical and analytical skills â¢ Pos...,good logical analytical skill positive attitud...,13.65,9,8
5,Testing,EDUCATION\nDINESH SAI PAPPURU\n+1(857) 313-502...,education dinesh sai pappuru boston github lin...,10.30,10,1


In [ ]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.5259235217826549